In [2]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(api_key=openai.api_key)

try:
    models = client.models.list()
    print("🔑 OpenAI API 정상 인증 완료 ✅")
except Exception as e:
    print("❌ 인증 실패:", e)

🔑 OpenAI API 정상 인증 완료 ✅


In [3]:
# 0단계: PDF 문서 로딩
import os
from langchain_community.document_loaders import PyPDFLoader

pdf_path = "./data/콘텐츠분쟁해결_사례.pdf"

print("📁 현재 작업 디렉토리:", os.getcwd())
print("📄 data 폴더 내 파일 목록:", os.listdir("./data"))

if os.path.exists(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print(f"✅ {len(documents)}개 페이지 로드 완료")
else:
    print("❌ PDF 파일 경로가 올바르지 않습니다.")

📁 현재 작업 디렉토리: c:\Users\hun\Desktop\llm_example
📄 data 폴더 내 파일 목록: ['ai-terminology.txt', 'ai_technologies.csv', 'finance-terminology.txt', 'taxinfo.txt', 'titanic.csv', 'tutorial-korean.pdf', '음식_recommendations.csv', '콘텐츠분쟁해결_사례.pdf']
✅ 109개 페이지 로드 완료


In [4]:
# 1단계: 문서 분할
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】", "\n【쟁점사항】", "\n【처리경위】", "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = text_splitter.split_documents(documents)
print(f"🧩 분할된 청크 수: {len(chunks)}")

🧩 분할된 청크 수: 104


In [5]:
# 2단계: 임베딩 설정
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)


In [ ]:
# 3단계: 벡터스토어 및 검색기 설정
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(chunks[:10], embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

"""RateLimitError: Error code: 429
"You exceeded your current quota" → 사용 가능한 OpenAI API **쿼터(요금제 한도)**를 초과했습니다."
이 초과로 인해 문제를 풉니다 ㅠㅠ""

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
# 4단계: LLM 설정
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    max_tokens=2000
)

In [12]:
# 5단계: 프롬프트 템플릿 작성
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
""")

In [14]:
# 6단계: QA 체인 생성
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True
)

NameError: name 'retriever' is not defined

In [15]:
# 7단계: 테스트 질문 작성
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

In [16]:
# 8단계: 분쟁 유형 분류 함수 (선택)
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"

In [17]:
response = qa_chain.run("온라인 게임 아이템이 사라졌는데 환불받을 수 있나요?")
print(response)

NameError: name 'qa_chain' is not defined